# Python sdk as Service Principal

## Overview

Details related to Databricks sdk for python are described: <a href="https://learn.microsoft.com/en-us/azure/databricks/dev-tools/sdk-python" target="_blank">here</a>

## Step 1: databricks sdk install

In [0]:
%pip install databricks-sdk --upgrade

In [0]:
dbutils.library.restartPython()

In [0]:
%pip show databricks-sdk | grep -oP '(?<=Version: )\S+'

## Step 2: Authentication
Details related to python authentication as Service Principal are described: <a href="https://learn.microsoft.com/en-us/azure/databricks/dev-tools/auth/oauth-m2m#python" target="_blank">here</a>

In [0]:
workspace_url = spark.conf.get("spark.databricks.workspaceUrl")
display(workspace_url)

### Manual Authentication Setup

In [0]:
dbutils.widgets.text(name='client_id', defaultValue='')
dbutils.widgets.text(name='token', defaultValue='')
dbutils.widgets.text(name='account_id', defaultValue='')

import os

sp_cid = dbutils.widgets.get('client_id')
sp_tkn = dbutils.widgets.get('token')
ws_url =  workspace_url
ac_url =  "https://accounts.azuredatabricks.net"
ac_id = dbutils.widgets.get('account_id')

In [0]:
display("sp_cid: " + sp_cid)
display("sp_tkn: " + sp_tkn)
display("ws_url: " + ws_url)
display("ac_url: " + ac_url)
display("ac_id: "  + ac_id)

### Auto Authentication Setup via Secrets

In [0]:
scopes = dbutils.secrets.listScopes()
scopes

In [0]:
secrets = dbutils.secrets.list("scp-ag83-cm-consumer-001")
secrets

In [0]:
dbutils.widgets.text(name='account_id', defaultValue='')

import os

sp_cid = dbutils.secrets.get(scope = "scp-ag83-cm-consumer-001", key = "scr-ag83-cm-consumer-admin-001-uuid")
sp_tkn = dbutils.secrets.get(scope = "scp-ag83-cm-consumer-001", key = "scr-ag83-cm-consumer-admin-001-token")
ws_url =  workspace_url
ac_url =  "https://accounts.azuredatabricks.net"
ac_id = dbutils.widgets.get('account_id')

In [0]:
display("sp_cid: " + sp_cid)
display("sp_tkn: " + sp_tkn)
display("ws_url: " + ws_url)
display("ac_url: " + ac_url)
display("ac_id: "  + ac_id)

### Account-level operations

In [0]:
from databricks.sdk import AccountClient

a = AccountClient(
  host          = ac_url,
  account_id    = ac_id,
  client_id     = sp_cid,
  client_secret = sp_tkn
)

### Workspace-level operations

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient(
  host          = ws_url,
  client_id     = sp_cid,
  client_secret = sp_tkn
)

## Step 3: Use the Sdk - Marketplace API

### Providers

In [0]:
cp_list = list(w.consumer_providers.list())
display(len(cp_list))

In [0]:
import json

for cp in list(w.consumer_providers.list()):
    print(json.dumps(cp.as_dict(), indent=2, ensure_ascii=False))

In [0]:
cp_dicts = [cp.as_dict() for cp in w.consumer_providers.list()]
df = spark.createDataFrame(cp_dicts)
display(df)

In [0]:
df.createOrReplaceTempView("provider")

In [0]:
%sql
select * 
from provider
where name = 'Databricks'


### Listings

In [0]:
cl_list = list(w.consumer_listings.list())
display(len(cl_list))

In [0]:
import json

for cl in list(w.consumer_listings.list())[:2]:
    print(json.dumps(cl.as_dict(), indent=2, ensure_ascii=False))

In [0]:
import json

cl_dicts = [
    {
        **cl.as_dict(),
        "detail": json.dumps(cl.as_dict().get("detail"), ensure_ascii=False),
        "summary": json.dumps(cl.as_dict().get("summary"), ensure_ascii=False)
    }
    for cl in w.consumer_listings.list()
]
df = spark.createDataFrame(cl_dicts)
display(df)

In [0]:
from pyspark.sql.types import (
    StructType, StructField, StringType, ArrayType, DoubleType, LongType, MapType
)

detail_schema = StructType([
    StructField("assets", ArrayType(StringType()), True),
    StructField("collection_date_end", LongType(), True),
    StructField("collection_date_start", LongType(), True),
    StructField("collection_granularity", StructType([
        StructField("interval", LongType(), True),
        StructField("unit", StringType(), True)
    ]), True),
    StructField("cost", StringType(), True),
    StructField("data_source", StringType(), True),
    StructField("description", StringType(), True),
    StructField("documentation_link", StringType(), True),
    StructField("embedded_notebook_file_infos", ArrayType(StructType([
        StructField("created_at", LongType(), True),
        StructField("display_name", StringType(), True),
        StructField("download_link", StringType(), True),
        StructField("file_parent", StructType([
            StructField("file_parent_type", StringType(), True),
            StructField("parent_id", StringType(), True)
        ]), True),
        StructField("id", StringType(), True),
        StructField("marketplace_file_type", StringType(), True),
        StructField("mime_type", StringType(), True),
        StructField("status", StringType(), True),
        StructField("status_message", StringType(), True),
        StructField("updated_at", LongType(), True)
    ])), True),
    StructField("file_ids", ArrayType(StringType()), True),
    StructField("geographical_coverage", StringType(), True),
    StructField("license", StringType(), True),
    StructField("pricing_model", StringType(), True),
    StructField("privacy_policy_link", StringType(), True),
    StructField("size", DoubleType(), True),
    StructField("support_link", StringType(), True),
    StructField("tags", ArrayType(StructType([
        StructField("tag_name", StringType(), True),
        StructField("tag_values", ArrayType(StringType()), True)
    ])), True),
    StructField("terms_of_service", StringType(), True),
    StructField("update_frequency", StructType([
        StructField("interval", LongType(), True),
        StructField("unit", StringType(), True)
    ]), True)
])

summary_schema = StructType([
    StructField("categories", ArrayType(StringType()), True),
    StructField("created_at", LongType(), True),
    StructField("created_by", StringType(), True),
    StructField("created_by_id", LongType(), True),
    StructField("exchange_ids", ArrayType(StringType()), True),
    StructField("git_repo", StructType([
        StructField("git_repo_url", StringType(), True)
    ]), True),
    StructField("listingType", StringType(), True),
    StructField("name", StringType(), True),
    StructField("provider_id", StringType(), True),
    StructField("provider_region", StructType([
        StructField("cloud", StringType(), True),
        StructField("region", StringType(), True)
    ]), True),
    StructField("published_at", LongType(), True),
    StructField("published_by", StringType(), True),
    StructField("setting", StructType([
        StructField("visibility", StringType(), True)
    ]), True),
    StructField("share", StructType([
        StructField("name", StringType(), True),
        StructField("type", StringType(), True)
    ]), True),
    StructField("status", StringType(), True),
    StructField("subtitle", StringType(), True),
    StructField("updated_at", LongType(), True),
    StructField("updated_by", StringType(), True),
    StructField("updated_by_id", LongType(), True)
])

schema = StructType([
    StructField("id", StringType(), True),
    StructField("detail", detail_schema, True),
    StructField("summary", summary_schema, True)
])

cl_dicts = [
    {
        "id": cl.as_dict().get("id"),
        "detail": cl.as_dict().get("detail"),
        "summary": cl.as_dict().get("summary")
    }
    for cl in w.consumer_listings.list()
]

df = spark.createDataFrame(cl_dicts, schema=schema)
display(df)

In [0]:
df.createOrReplaceTempView("listing")

In [0]:
%sql
select *
from listing
where summary.provider_id = 'ea1e69ff-0127-4c94-bf39-e841fe1d19d2'
  and contains(summary.name, 'Simulated Retail Customer Data')


### Fullfilments

In [0]:
cf_list = list(w.consumer_fulfillments.list(listing_id='a82597f6-5ada-49d5-b934-d6c9dece16a1'))
display(len(cf_list))

In [0]:
import json

for cf in list(w.consumer_fulfillments.list(listing_id='a82597f6-5ada-49d5-b934-d6c9dece16a1')):
    print(json.dumps(cf.as_dict(), indent=2, ensure_ascii=False))

In [0]:
import json

for cf in list(w.consumer_fulfillments.get(listing_id='a82597f6-5ada-49d5-b934-d6c9dece16a1')):
    print(json.dumps(cf.as_dict(), indent=2, ensure_ascii=False))

### Installations

In [0]:
import pandas as pd

ids = [row['id'] for row in df.select("id").collect()]
results = []

for listing_id in ids:
    try:
        fulfillments = w.consumer_fulfillments.get(listing_id=listing_id)
        for f in fulfillments:
            d = f.as_dict()
            d['listing_id'] = listing_id
            results.append(d)
    except Exception as e:
        results.append({'listing_id': listing_id, 'error': str(e)})

result_df = spark.createDataFrame(pd.DataFrame(results))
display(result_df)

In [0]:
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType

ids = [row['id'] for row in df.filter(df['summary.provider_id'] == "ea1e69ff-0127-4c94-bf39-e841fe1d19d2").select("id").collect()]
results = []

for listing_id in ids:
    try:
        fulfillments = w.consumer_fulfillments.get(listing_id=listing_id)
        for f in fulfillments:
            d = f.as_dict()
            d['listing_id'] = listing_id
            results.append(d)
    except Exception as e:
        results.append({'listing_id': listing_id, 'error': str(e)})

if results:
    result_df = spark.createDataFrame(pd.DataFrame(results))
else:
    schema = StructType([
        StructField("data_object_type", StringType(), True),
        StructField("name", StringType(), True),
        StructField("listing_id", StringType(), True)
    ])
    result_df = spark.createDataFrame([], schema)

display(result_df)
result_df.printSchema()

In [0]:
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType

ids = [row['id'] for row in df.select("id").collect()]
results = []

for listing_id in ids:
    try:
        fulfillments = w.consumer_fulfillments.get(listing_id=listing_id)
        for f in fulfillments:
            d = f.as_dict()
            d['listing_id'] = listing_id
            results.append(d)
    except Exception as e:
        results.append({'listing_id': listing_id, 'error': str(e)})

if results:
    result_df = spark.createDataFrame(pd.DataFrame(results))
else:
    schema = StructType([
        StructField("data_object_type", StringType(), True),
        StructField("name", StringType(), True),
        StructField("listing_id", StringType(), True)
    ])
    result_df = spark.createDataFrame([], schema)

display(result_df)
result_df.printSchema()

In [0]:
result_df.createOrReplaceTempView("asset")

In [0]:
%sql
select l.id, 
       count(*) as num_asset
from asset a
inner join listing l
 on a.listing_id = l.id
group by l.id
order by num_asset desc

In [0]:
%sql
select *
from listing
where id = '1418c785-a283-4b73-b878-f4f2d49945fa'

In [0]:
# Crea lo schema marketplace nel catalogo platform_catalog
spark.sql("CREATE SCHEMA IF NOT EXISTS platform_catalog.marketplace")

# Salva le tre tabelle come managed table nello schema marketplace
spark.sql("CREATE OR REPLACE TABLE platform_catalog.marketplace.asset AS SELECT * FROM asset")
spark.sql("CREATE OR REPLACE TABLE platform_catalog.marketplace.listing AS SELECT * FROM listing")
spark.sql("CREATE OR REPLACE TABLE platform_catalog.marketplace.provider AS SELECT * FROM provider")

In [0]:
%sql

USE CATALOG platform_catalog;
USE SCHEMA marketplace;

select          p.name              as provider_name,
                l.summary.name      as listing_name, 
                count(*)            as num_asset
from            asset   a
inner join      listing l
    on          a.listing_id = l.id
inner join      provider p
    on          l.summary.provider_id = p.id
group by        p.name,
                l.summary.name
order by        num_asset desc